<a href="https://colab.research.google.com/github/krMaynard/genai/blob/main/gemini_image_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Standard library imports
import base64
import random
import textwrap
from io import BytesIO

# Third-party imports
from google import genai
from google.colab import files
from google.colab import userdata
from google.genai import types
from IPython.display import display, Markdown
from PIL import Image

secret_api_key = userdata.get('GOOGLE_API_KEY')

client = genai.Client(api_key=secret_api_key)

prompts = []
text_responses = []
image_filenames = []

In [9]:
prompt = textwrap.dedent("""\
    Generate a hyperrealistic, cinematic photograph of a surreal moment in a modern
    Seoul subway station. A full-grown, elegant giraffe is sitting upright
    on a polished metal bench, impeccably dressed in a tailored dark grey
    business suit with a crisp white shirt and a silk tie. Next to the
    giraffe sits a Korean teenage boy in his late teens, wearing
    contemporary streetwear (a stylish hoodie, jeans, and sneakers). The
    boy is turned towards the giraffe, talking to it with an earnest and
    animated expression, gesturing with one hand. The giraffe appears to
    be listening intently.

    The subway station is clean and brightly lit with a mix of cool
    fluorescent overheads and warm accent lights. In the background,
    digital signs display train information in glowing Hangul (Korean
    script). The tiled walls and floor are pristine, reflecting the
    lights. The yellow safety line is visible on the edge of the platform.

    Capture the scene with a shallow depth of field, keeping the giraffe
    and the boy in sharp focus while the background is softly blurred. The
    lighting should create soft shadows, enhancing the realism and the
    quiet, intimate nature of their bizarre conversation.

    **Style:** Photorealistic, 8K resolution, ultra-detailed, sharp focus,
    shot on a Sony A7R IV with a 50mm f/1.2 lens.\
""")

def generate_image(prompt):
  prompts.append(prompt)

  response = client.models.generate_content(
      model="gemini-2.0-flash-preview-image-generation",
      contents=prompt,
      config=types.GenerateContentConfig(
        response_modalities=['TEXT', 'IMAGE']
      )
  )

  for part in response.candidates[0].content.parts:
    if part.text is not None:
      text_response = part.text
      text_responses.append(text_response)
    elif part.inline_data is not None:
      image = Image.open(BytesIO((part.inline_data.data)))
      random_number = random.randint(0, 1000)
      image_filename = f'gemini-native-image_{random_number}.png'
      image_filenames.append(image_filename)
      image.save(image_filename)
      image.show()
  return response, text_response, image_filename


In [10]:
for i in range (10):
  imagen = generate_image(prompt)
  text_response = imagen[1]
  image_filename = imagen[2]
  display(Markdown(f'**Response**: {text_response}'))
  print(f'Image saved to {image_filename}')

**Response**: I will generate a hyperrealistic cinematic photograph of a surreal scene within a modern Seoul subway station. The image will feature a sophisticated, adult giraffe seated upright on a shiny metal bench, wearing a perfectly fitted dark grey business suit, a white dress shirt, and a silk tie. Beside it will be a Korean teenage boy, dressed in fashionable streetwear including a hoodie, jeans, and sneakers, actively conversing with the giraffe. The boy will be looking at the giraffe with a sincere and expressive face, gesturing with one hand as if emphasizing a point. The giraffe will seem to be attentively listening.

The subway station environment will be depicted as immaculate and well-lit, with a combination of cool fluorescent ceiling lights and warmer accent lighting. Train schedule information in bright Hangul characters will be visible on digital displays in the blurred background. The station's tiled walls and floor will be spotless, subtly reflecting the ambient light. The yellow safety line along the platform edge will be clearly visible. The photograph will employ a shallow depth of field, ensuring that the giraffe and the boy are sharply in focus, while the background elements are softly out of focus. Soft shadows will be cast throughout the scene, contributing to the photorealistic quality and the quiet, almost private atmosphere of this unusual exchange. The image will be rendered in 8K resolution with ultra-fine detail and sharp focus, as if captured with a Sony A7R IV camera using a 50mm f/1.2 lens.


Image saved to gemini-native-image_15.png


**Response**: I will generate a hyperrealistic photograph depicting a surreal scene in a modern Seoul subway station. A refined adult giraffe, wearing a dark grey tailored business suit, a white shirt, and a silk tie, is seated upright on a shiny metal bench. Beside it, a Korean teenage boy in stylish streetwear (hoodie, jeans, sneakers) is animatedly talking to the giraffe, gesturing with one hand. The giraffe is attentively listening. The bright and clean subway station features a mix of cool and warm lighting, digital signs displaying glowing Hangul train information in the blurred background, and pristine tiled walls and floor with a visible yellow safety line. The image will have a shallow depth of field, focusing sharply on the giraffe and the boy, with soft shadows enhancing the realism of this quiet, bizarre interaction. The photograph will be in photorealistic style, 8K resolution, ultra-detailed, with sharp focus, as if shot on a Sony A7R IV with a 50mm f/1.2 lens.


Image saved to gemini-native-image_334.png


**Response**: I will generate a hyperrealistic cinematic photograph of a surreal moment in a modern Seoul subway station. A distinguished giraffe, perfectly attired in a dark grey tailored business suit, a white shirt, and a silk tie, is seated upright on a sleek metal bench. Beside the giraffe, a Korean teenage boy in stylish streetwear (hoodie, jeans, and sneakers) is engaged in an animated conversation, gesturing expressively. The giraffe listens attentively. The clean, brightly lit subway station features cool fluorescent and warm accent lighting, digital signs displaying glowing Hangul train information in the blurred background, and pristine tiled walls and floor reflecting the light. A yellow safety line marks the platform edge. The scene is captured with a shallow depth of field, emphasizing the sharply focused giraffe and boy, with soft shadows enhancing the realism and intimate atmosphere of their unusual exchange. The style will be photorealistic, in 8K resolution, ultra-detailed, sharp focus, as if shot on a Sony A7R IV with a 50mm f/1.2 lens.


...

Image saved to gemini-native-image_952.png


**Response**: I will generate a hyperrealistic photograph of a surreal scene in a modern Seoul subway station. A refined adult giraffe, dressed in a sharp dark grey suit, white shirt, and silk tie, will be seated upright on a shiny metal bench. Beside it, a Korean teenage boy in stylish streetwear will be animatedly talking and gesturing towards the giraffe, who will be shown listening attentively. The clean, brightly lit station will feature digital signs with glowing Hangul in the blurred background, along with pristine tiled walls and a yellow safety line. The shallow depth of field will focus on the giraffe and the boy, with soft shadows enhancing the realism and the intimate atmosphere of their unusual interaction.



Image saved to gemini-native-image_232.png


**Response**: I will generate a hyperrealistic image depicting a surreal scene within a modern Seoul subway station. A refined giraffe, wearing a dark grey business suit, white shirt, and silk tie, is seated upright on a shiny metal bench. Beside it, a late-teen Korean boy in stylish streetwear (hoodie, jeans, sneakers) is turned towards the giraffe, gesturing with one hand as he speaks with an earnest and animated expression. The giraffe is attentive, seemingly listening. The clean, brightly lit station features cool fluorescent and warm accent lighting, digital signs displaying train information in glowing Hangul, and pristine tiled walls and floor that reflect the light. A yellow safety line is visible. The image will have a shallow depth of field, focusing sharply on the giraffe and boy with a softly blurred background. Soft shadows will enhance the realism and intimate feel of this bizarre conversation. The style will be photorealistic, 8K resolution, ultra-detailed, with sharp focus, as if shot on a Sony A7R IV with a 50mm f/1.2 lens.



Image saved to gemini-native-image_871.png


In [11]:
# Download all local files

for filename in image_filenames:
  files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
# Delete all local files

!rm *.*